# Run Q3Dfit from this Jupyter Notebook in Astroconda Environment.

This Jupyter notebook allows you to run Q3Dfit, a PSF decomposition and spectral analysis package tailored for JWST NIRSpec and MIRI IFU observations. 

Q3Dfit is developed as a science-enabling data product by the Early Release Science Team #1335 Q3D. You can find more information about this ERS program **Q3D** [here](https://wwwstaff.ari.uni-heidelberg.de/dwylezalek/q3d.html) and [here](https://www.stsci.edu/jwst/science-execution/approved-programs/dd-ers/program-1335).

The software is based on the existing package IFSFIT developed by Dave Rupke (see [ADS link](https://ui.adsabs.harvard.edu/abs/2017ApJ...850...40R/abstract)).

The following notebook will guide you through the initialization procedure and will then perform the analysis. 

## Table of Contents

* [1. Initialization](#chapter1)
    * [1.1. Setting up the directory tree](#chapter1_1)
    * [1.2. Setting up the fitting parameters](#chapter1_2)
        * [1.2.3. Line ratio constraints](#chapter1_2_3)
    * [1.3. Setting up the plotting parameters](#chapter1_3)
* [2. Run Q3Dfit](#chapter2)
* [3. Run plotting script (q3da)](#chapter3)

## 1. Initialization <a class="anchor" id="chapter1"></a>

In [ ]:
import os.path
import numpy as np

In [ ]:
# Convenience function
# Ref: https://github.com/spacetelescope/jwebbinar_prep/blob/main/imaging_mode/imaging_mode_stage_1.ipynb
from astropy.utils.data import download_file
def download_files(files, output_directory, force=False):
    """Given a tuple or list of tuples containing (URL, filename),
    download the given files into the current working directory.
    Downloading is done via astropy's download_file. A symbolic link
    is created in the specified output dirctory that points to the
    downloaded file.
    
    Parameters
    ----------
    files : tuple or list of tuples
        Each 2-tuple should contain (URL, filename), where
        URL is the URL from which to download the file, and
        filename will be the name of the symlink pointing to
        the downloaded file.
        
    output_directory : str
        Name of the directory in which to create the symbolic
        links to the downloaded files
        
    force : bool
        If True, the file will be downloaded regarless of whether
        it is already present or not.
                
    Returns
    -------
    filenames : list
        List of filenames corresponding to the symbolic links
        of the downloaded files
    """
    # In the case of a single input tuple, make it a
    # 1 element list, for consistency.
    filenames = []
    if isinstance(files, tuple):
        files = [files]
        
    for file in files:
        filenames.append(file[1])
        if force:
            print('Downloading {}...'.format(file[1]))
            demo_file = download_file(file[0], cache='update')
            # Make a symbolic link using a local name for convenience
            if not os.path.islink(os.path.join(output_directory, file[1])):
                os.symlink(demo_file, os.path.join(output_directory, file[1]))
        else:
            if not os.path.isfile(os.path.join(output_directory, file[1])):
                print('Downloading {}...'.format(file[1]))
                demo_file = download_file(file[0], cache=True)
                # Make a symbolic link using a local name for convenience
                os.symlink(demo_file, os.path.join(output_directory, file[1]))
            else:
                print('{} already exists, skipping download...'.format(file[1]))
                continue
    return filenames    

### 1.1. Setting up the directory tree <a class="anchor" id="chapter1_1"></a>

Define the output directories. We recommend creating a working directory that you name after your target, in which all outputs from Q3Dfit will be saved. 

In [ ]:
# book-keeping variables (optional; for this box only)
volume = 'nirspec-etc/'
gal = 'nirspec-etc' # prefix label for output files

# Output directories/files
outdir = volume+'' # path + name of directory for saving the output files
path_to_npy_file = volume+'' # path for initproc.npy
logfile = outdir+gal+'_fitlog.txt'

In [ ]:
# make subdirectory to hold data
if not os.path.exists(gal):
    os.makedirs(gal)
    
infile_tup=('https://rhodes.box.com/shared/static/ob0l4nzlzw5aif0l7700bywxh115hd9w.fits','nirspec-etc-simulation.fits')
infile = download_files(infile_tup, outdir, force=True)[0]
# add subdirectory to filenames
infile = outdir+infile

Some general information about your cube:

argsreadcube is a dictionary of attributes sent to the CUBE class

In [ ]:
argsreadcube = {'wmapext': None,
                'waveunit_in': 'micron',
                'fluxunit_in': ''}

if not os.path.isfile(infile): 
    print('Data cube not found.')
else:
    from q3dfit.common.readcube import CUBE
    cube = CUBE(infile=infile, **argsreadcube)
    ncols = cube.ncols # number of columns
    nrows = cube.nrows # number of rows

Wavelength range over which to fit data and the range of wavelength to ignore:

In [ ]:
fitrange = [0.98,1.80] #in um
cutrange = np.array([1.4133,1.4743])


Generating the quasar template used for PSF subtraction:


In [ ]:
from q3dfit.common import makeqsotemplate
qsotemplate = volume+'nirspec_ETC_QSO.npy' 
makeqsotemplate.makeqsotemplate(infile,qsotemplate,waveunit_out='micron',wmapext=None)

### 1.2. Setting up the fitting parameters <a class="anchor" id="chapter1_2"></a>

What lines do you want to fit? You can choose from the linelist <font color='red'>provide reference here</font>.

In [ ]:
lines = ['Halpha', 'Hbeta', '[OIII]4959', '[OIII]5007']
zsys_gas = 1.489

How many components do you want to be fitted to the emission lines?

In [ ]:
maxncomp = 1

In [ ]:
# Initialize line ties, n_comps, z_inits, and sig_inits.
linetie = dict()
ncomp = dict()
zinit_gas = dict()
siginit_gas = dict()
for i in lines:
    linetie[i] = 'Halpha'
    ncomp[i] = np.full((ncols,nrows),maxncomp)
    zinit_gas[i] = np.full((ncols,nrows,maxncomp),1.489)
    siginit_gas[i] = np.full(maxncomp,50.)

`siglim_gas` sets lower and upper bounds for the Gaussian width (sigma) of the emission line. These limits can be set globablly, for all spaxels and components, by defining a 2-element array. The limits can also be set for individual spaxels (but all components) by defining an (Ncol x Nrow x 2) array.

In [ ]:
# Global limit
siglim_gas = np.array([5., 500.])

# Spaxel-by-spaxel limit
# siglim_gas = np.ndarray((dx, dy, 2))
# siglim_gas[:,:,] = array([5.,1000.])
# siglim_gas[13, 10, :] = array([5.,500.])

Description of continuum fitting parameters:
- `hostord` = [Optional] additive Legendre polynomial for host [starlight] component. Adds components up to order `hostord`. 0 < hostord < 4
- `qsoord` = [Optional] additive Legendre polynomial for host [starlight] component. Adds components up to order `qsoord`. 0 < hostord < 4

In [ ]:
fcncontfit = 'fitqsohost'
argscontfit = dict()
#argscontfit['blrpar'] = [0., 1.6400, 0.5000/299792.*1.6400,
#                         0., 1.2150, 0.5000/299792.*1.2150]
argscontfit['qsoxdr'] = qsotemplate
argscontfit['qsoord'] = 10
argscontfit['hostord'] = 10
#argscontfit['refit'] = 'ppxf'

#### 1.2.3. Line ratio constraints <a class="anchor" id="chapter1_2_3"></a>
`line1`, `line2`, and `comp` are required. `value` is the initial value of `line1`/`line2`, and `lower` and `upper` limits can also be specified. (If they are not, and the line pair is a doublet in the doublets.tbl file, then the lower and upper limits are set using the data in that file.) The ratio can also be `fixed`.

In [ ]:
# Required columns:
#line1 = ['[NI]5198']
#line2 = ['[NI]5200']
#comp = np.array([0], dtype=np.int32)

# Optional columns:
#value = [1.5]
#fixed = [True]
# lower = []
# upper = []

# Write table
#from astropy.table import QTable
#lineratio = QTable([line1, line2, comp, value, fixed], names=['line1', 'line2', 'comp', 'value', 'fixed'])

### 1.3. Setting up the plotting parameters <a class="anchor" id="chapter1_3"></a>

One dictionary must be defined for a line plot to appear. `argspltlin1` holds the options for the first line plot, and `argspltlin2` is for the second. Parameter options are fed in as keyword/data pairs.

Required keywords in each dictionary:
* `nx`: Number of subplots in the horizontal direction
* `ny`: Number of subplots in the vertical direction
* Options for centerting the plot:
    - `line`: a string list of line labels
    - `center_obs`: a float list of wavelengths of each subplot center, in the observed (plotted) frame
    - `center_rest`: a float list of wavelengths of each subplot center, in the rest frame, which are converted to obs. frame

Optional keywords:
* `size`: float list of widths in wavelength space of each subplot; if not specified, default is 300 $Å$
* `IR`: set to `True` to use infrared-style plot

In [ ]:
argspltlin1 = {'nx': 3,
               'ny': 1,
               'line': ['Hbeta', '[OIII]5007', 'Halpha'],
               'size': [0.05, 0.05, 0.05]}

argscontplot = {'xstyle': 'lin',
                'ystyle': 'lin',
                'fluxunit_in': 'flambda',
                'fluxunit_out': 'flambda',
                'mode': 'dark'}

The following cell will create the initialisation structure. **Do not change anything in this cell.** 

In [ ]:
q3di = { \
        # Required pars
        'fitran': fitrange,
        'fluxunits': 1,
        'infile': infile,
        'label': gal,
        'lines': lines,
        'linetie': linetie,
        'maxncomp': maxncomp,
        'name': gal,
        'ncomp': ncomp,
        'outdir': outdir,
        'zinit_gas': zinit_gas,
        'zsys_gas': zsys_gas,
        # Optional pars
        'argscheckcomp': {'sigcut': 3},
        'fcncontfit': fcncontfit,
        'argscontfit': argscontfit,
        'argscontplot': argscontplot,
        'argsreadcube': argsreadcube,
        'argspltlin1': argspltlin1,
        #'decompose_qso_fit': 1,
        #'remove_scattered': 1,
        'fcncheckcomp': 'checkcomp',
        'maskwidths_def': 500.,
        'emlsigcut': 2,
        'logfile': logfile,
        'siglim_gas': siglim_gas,
        'siginit_gas': siginit_gas,
        'cutrange': cutrange,
        'nocvdf': 1,
    }

If you want to run `q3dfit` in batch mode, run this cell, which saves initproc to an `npy` file. In your python command line, read in that `npy` file with
<pre><code>import numpy as np
initproc = np.load('path/to/the/npy/file/initproc.npy',allow_pickle=True)</code></pre>
and then run `q3dfit` with 
<pre><code>from q3dfit.common.q3df
import q3df q3df(initproc[()],cols=cols,rows=rows)</code></pre>

In [ ]:
q3di_npy = 'q3di.npy'
np.save(path_to_npy_file+q3di_npy,q3di)

## 2. Run Q3Dfit <a class="anchor" id="chapter2"></a>

In [ ]:
# This may be unique to the user, insert your path to the q3dfit/ folder here
import sys
if '../../' not in sys.path:
    sys.path.append('../../')

from q3dfit.common.q3df import q3df
from q3dfit.common.q3da import q3da

In [ ]:
cols = 13
rows = 20

In [ ]:
q3df(q3di,cols=cols,rows=rows)

## 3. Run plotting script (q3da) <a class="anchor" id="chapter3"></a>

Inline plotting:

In [ ]:
%matplotlib notebook
q3da(q3di,cols=cols,rows=rows)

Plots in an external, interactive window:

In [ ]:
%matplotlib
q3da(q3di,cols=cols,rows=rows)